In [1484]:
import math
import random

# A minimal autodiff engine
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value({self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self.data * -1 

    def __sub__(self,other):
        out = Value(self.data + (-other.data),(self,other),'-')
        return out
        

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other): 
        return self + other
        
    def __rmul__(self, other): return self * other
    def __neg__(self): return self * -1
    def __sub__(self, other): return self + (-other)
    def __rsub__(self, other): return other + (-self)
    def __truediv__(self, other): return self * other**-1
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Only supporting int/float powers"
        out = Value(self.data ** other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other - 1)) * out.grad
        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()

        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)
        build(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [1486]:
#Input values
x1 = Value(2.0)
x2 = Value(3.0)
#Weights
w1 = Value(-3.0)
w2 = Value(2.0)
#Bias
b = Value(1)

x1w1 = x1 * w1
x2w2 = x2 * w2
x1w1x2w2 = x1w1 + x2w2
y = x1w1x2w2 + b
z = y.tanh()
z

Value(0.7615941559557649)

In [1488]:
z.grad = 1
z.backward()

In [1490]:
y.grad

0.41997434161402614

In [1492]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(1.0)

    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()

    def params(self):
        return [self.b] + self.w
        
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        out = [neuron(x) for neuron in self.neurons]
        return out if len(out) > 1 else out[0]

    def params(self):
        params = []
        for neuron in self.neurons:
            params.extend(neuron.params())

        return params

class MLP:
    def __init__(self, nin, nouts):
        sizes = [nin] + nouts
        self.layers = [Layer(sizes[i], sizes[i + 1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def params(self):
        params = []
        for layer in self.layers:
            params.extend(layer.params())

        return params

mlp = MLP(3, [4, 4, 1])

In [1494]:
xs = [[2, 3, -1], [3, -1, 0.5], [0.5, 1, 1], [1, 1, -1]]
ys = [1.0, -1.0, -1.0, 1.0]
ys_val = [Value(y) for y in ys]

In [1496]:
#mlp.params()
y_preds = [mlp([Value(xi) for xi in x]) for x in xs]
loss = sum((y_pred - y_true)**2 for y_pred, y_true in zip(y_preds, ys_val))
loss

Value(4.6230394766078895)

In [1498]:
loss.backward()

In [1500]:
for p in mlp.params():
    p.data -= 0.01 * p.grad

In [1502]:
y_preds

[Value(0.8118385788410258),
 Value(0.9021864588180127),
 Value(-0.04292516981059815),
 Value(0.7690688381732776)]